# Projeto 2 - Ciência dos Dados

Nome:Luís Filipe Loureiro

Nome: João Bueno


___
## Classificador automático de sentimento do modelo Golf da Volkswagen


## Introdução

O presente estudo tem como objetivo criar um machine learning que, por meio do Teorema probabilístico de Bayes, consegue classificar um tweet quanto a sua relevância. O foco de estudo foi o carro Golf da marca Volkswagen, tentou-se criar um modelo que identifica aquelas mensagens que revelam a visão dos consumidores sobre o produto, seja no sentido de criticá-lo, seja no sentido de manifestar um desejo de adquiri-lo ou vendê-lo.

Para isso foi importado 750 tweets por meio de uma conta developer no Twitter e analisado um por um quanto ao sua relevância. O critério de exploração foi o seguinte: o tweet recebeu valor 0, ou seja, irrelevante, quando se referia ao esport golf; recebeu o valor 1, ou seja, neutro aquele relacionado ao carro, mas que não exprimia qualquer juízo de valor e recebeu o valor 2, ou seja, relevante aquele que expunha uma visão do mercado sobre o produto. Para deixar mais claro o critério de avaliação, apresenta-se exemplos de tweets avaliados:
    - "Queria jogar golf", classificado como irrelevante [0]
    - "Trouxe o golf lá da casa do Victor", classificado como neutro [1]
    - "Estou apaixonada pelo golf que vão lançar", classificado como relevante [2]
    
Dos 750 tweets importados, 500 foram utilizados para identificação de um padrão por meio do Teorema de Bayes e nos 250 restantes foi aplicado o machine learning e comparado os resultados.


## Importando as bibliotecas necessárias

In [ ]:
%%capture

#Instalando o tweepy
# !pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: @luisfloureiro1

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @luisfloureiro1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coleta de tweets e os salvando em um arquivo Excel

In [ ]:
#Produto escolhido:
produto = 'Golf'

#Quantidade mínima de mensagens capturadas:
n = 1600
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break


#Embaralhando as mensagens para reduzir um possível viés
shuffle(list(set(msgs)))



In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(list(set(msgs))[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(list(set(msgs))[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Filtrando as pontuação e espaços duplos dos tweets coletados



In [3]:
#Para não ser mais necessário conectar-se ao twitter, basta rodas essa cédula e a cédula que importa as bibliotecas
tweets= pd.read_excel('Golf.xlsx')


In [4]:
valores=[]

for valor in tweets.loc[:,"Relevancia"]:
    valores.append(valor)


In [5]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [6]:
clean_treinamento=[]
for tweet in tweets.loc[:,"Treinamento"]:
    clean_treinamento.append((cleanup(tweet)))

    
        
# clean_treinamento

In [7]:
tweets_clean =pd.DataFrame(clean_treinamento)
tweets_clean.loc[:,"Relevancia"]=1

In [8]:
#Adicionando a coluna relevancia
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"Relevancia"]=valores[numero]

tweets_clean.columns=["tweet","Relevancia"]


In [9]:
#Retirando double spaces
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"tweet"]=re.sub(' +', ' ',tweets_clean.loc[numero,"tweet"])
    tweets_clean.loc[numero,"tweet"]=tweets_clean.loc[numero,"tweet"].strip()


## Arrumando base de dados Treinamento para inserir no Teorema de Bayes

In [10]:
irrelevantes=tweets_clean.loc[tweets_clean.Relevancia==0]

neutros=tweets_clean.loc[tweets_clean.Relevancia==1]

relevantes=tweets_clean.loc[tweets_clean.Relevancia==2]

In [11]:
N=[]
for neutro in neutros.tweet:
    N.append(neutro)

N01=','.join(N)
serie_neutro=pd.Series(N01.split())
tabela_neutro=serie_neutro.value_counts()
tabela_neutro_relativa = serie_neutro.value_counts(True)

In [12]:
I=[]
for irrelevante in irrelevantes.tweet:
    I.append(irrelevante)
    
I01=','.join(I)
serie_irrelevante=pd.Series(I01.split())
tabela_irrelevante=serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)

tabela_irrelevante.sum()


4988

In [13]:
R=[]
for relevante in relevantes.tweet:
    R.append(relevante)
    
R01=','.join(R)
serie_relevante=pd.Series(R01.split())
tabela_relevante=serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)

In [14]:
total=N01+I01+R01
total2=len(tabela_neutro)+len(tabela_irrelevante)+len(tabela_relevante)

In [15]:
serie_Total=pd.Series(total.split())

In [16]:
tabela_Total_relativa = serie_Total.value_counts(True)

In [17]:
set_neutro= set(tabela_neutro_relativa.index)
set_irrelevante= set(tabela_irrelevante_relativa.index)
set_relevante= set(tabela_relevante_relativa.index)
int = set_neutro.intersection(set_irrelevante)

In [18]:
inter=set_relevante.intersection(int)
#inter

## Arrumando a base de dados Teste para ser inserida no Teorema de Bayes


In [19]:
xls = pd.ExcelFile('Golf.xlsx')
teste = pd.read_excel(xls, 'Teste')

In [20]:
clean_teste=[]
classificacao=[]

for valor in teste.loc[:,"Relevancia"]:
    classificacao.append(valor)

for tweet in teste.loc[:,"Teste"]:
    clean_teste.append((cleanup(tweet)))

teste_clean=pd.DataFrame(clean_teste)
teste_clean.loc[:,"Relevancia"]=1
for numero in np.arange(0,250):
    teste_clean.loc[numero,"Relevancia"]=classificacao[numero]

teste_clean.columns=["tweet","Relevancia"]


In [21]:
for numero in np.arange(0,250):
    tweets_clean.loc[numero,"tweet"]=re.sub(' +', ' ',tweets_clean.loc[numero,"tweet"])
    tweets_clean.loc[numero,"tweet"]=tweets_clean.loc[numero,"tweet"].strip()

a =pd.Series(teste_clean.loc[:,"tweet"].str.split())
lista1=[]
for elemento in a:
    for e in elemento:
        lista1.append(e)


___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [40]:
frase='golf quero um'
frase1=frase.lower()
frase2=cleanup(frase1.lower())
frase3=pd.Series(frase2.split())
lista_frase=list(frase3)

irrelevante=1
neutro=1
relevante=1

P_I=(len(I)/total2)
P_N=(len(N)/total2)
P_R=(len(R)/total2)

for word in lista_frase:
    
    if word not in tabela_irrelevante:
        irrelevante*=1/((total2+len(tabela_irrelevante)))
        
    else:
        irrelevante*=(tabela_irrelevante[word]+1)/(total2+len(tabela_irrelevante))
    
    if word not in tabela_neutro:
        neutro*=1/((total2+len(tabela_neutro)))
    else:
        neutro*=(tabela_neutro[word]+1)/(total2+len(tabela_neutro))
        
    if word not in tabela_relevante:
        relevante*=1/((total2+len(tabela_relevante)))
    else:
        relevante*=(tabela_relevante[word]+1)/(total2+len(tabela_relevante))
        
        

irrelevante*P_I
neutro*P_N
relevante*P_R
if irrelevante>neutro and irrelevante>relevante:
    print('Neutro') 
elif relevante>irrelevante and relevante>neutro:
    print('Relevante')
elif neutro>irrelevante and neutro>relevante:
    print('Irrelevante') 
    
print("Irrelevante {0}".format(irrelevante))
print("neutro {0}".format(neutro))
print("relevante {0}".format(relevante))

Relevante
Irrelevante 2.8262535777821616e-07
neutro 7.493520474786504e-08
relevante 9.486646600585076e-07


In [23]:
lista_frase

['golf', 'chegou', 'yaaaaaaaaah']

In [24]:
teste_clean.loc[:,"Teste"]=1
teste_clean.head()


,tweet,Relevancia,Teste
0,um mano acabou de espetar o seu golf no muro k...,1,1
1,eu amo o golf ♡ https t co ugqfnhyznh,2,1
2,esmuellert_ david_alaba ainda bem que não fa...,0,1
3,caramba tequion e os dramas q vc me prometeu m...,0,1
4,bazert eu vou jogar golf,0,1


In [32]:
teste_clean.loc[:,"tweet"].head()


0    um mano acabou de espetar o seu golf no muro k...
1              eu amo o golf ♡ https   t co ugqfnhyznh
2     esmuellert_  david_alaba ainda bem que não fa...
3    caramba tequion e os dramas q vc me prometeu m...
4                            bazert eu vou jogar golf 
Name: tweet, dtype: object

In [41]:
teste=[]


for frase in teste_clean.loc[:,"tweet"]:    
    frase1=frase.lower()
    frase3=pd.Series(frase1.split())
    lista_frase=list(frase3)
    irrelevante=1
    neutro=1
    relevante=1

    for word in lista_frase:


        if word not in tabela_irrelevante:
            irrelevante*=1/((total2+len(tabela_irrelevante)))

        else:
            irrelevante*=(tabela_irrelevante[word]+1)/(total2+len(tabela_irrelevante))

        if word not in tabela_neutro:
            neutro*=1/((total2+len(tabela_neutro)))
        else:
            neutro*=(tabela_neutro[word]+1)/(total2+len(tabela_neutro))

        if word not in tabela_relevante:
            relevante*=1/((total2+len(tabela_relevante)))
        else:
            relevante*=(tabela_relevante[word]+1)/(total2+len(tabela_relevante))

    P_I=(len(I)/total2)
    P_N=(len(N)/total2)
    P_R=(len(R)/total2)
    irrelevante*P_I
    neutro*P_N
    relevante*P_R
    
    if irrelevante>neutro and irrelevante>relevante:
        teste.append(0)
        
    elif relevante>irrelevante and relevante>neutro:
        teste.append(1)
    elif neutro>irrelevante and neutro>relevante:
        teste.append(2) 


In [42]:
teste

[2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [43]:
for numero in np.arange(1,250):
    teste_clean.loc[numero,"Teste"]=teste[numero]

In [44]:
teste_clean.head()

,tweet,Relevancia,Teste
0,um mano acabou de espetar o seu golf no muro k...,1,1
1,eu amo o golf ♡ https t co ugqfnhyznh,2,0
2,esmuellert_ david_alaba ainda bem que não fa...,0,0
3,caramba tequion e os dramas q vc me prometeu m...,0,0
4,bazert eu vou jogar golf,0,0


In [45]:
ct = pd.crosstab(teste_clean.Relevancia, teste_clean.Teste)
ct

Teste,0,1,2
Relevancia,,,
0,136,9,6
1,35,13,6
2,25,21,0


In [47]:
a=136+13
print(a/250)

0.596


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**